In [ ]:
%load_ext tikzmagic

In [ ]:
import tfo_parser_ulambda as ul
import IPython.display
import copy

g_test = True
g_parser = ul.parser()

def f_expx(txt):
    def f_parse(txt):
        return g_parser.parse(txt)
    def f_tree(parsed):
        transl = {'application':'appl', 'abstraction':'abstr'}
        id_cnt = [0]
        def next_id():
            id_cnt[0] = id_cnt[0]+1; return id_cnt[0]-1;
        def recurse(node):
            if type(node) == ul.lark.tree.Tree:
                if node.data not in ['variable', 'term']:
                    return {'id':next_id(), 'type':transl.get(node.data, node.data), 'kids':[recurse(x) for x in node.children]}
                else:
                    return recurse(node.children[0])
            else:
                return {'id':next_id(), 'type':'var', 'name' : str(node)}
        root = recurse(parsed)
        return root
    parsed = f_parse(txt)
    tree = f_tree(parsed)
    return {'txt':txt, 'parsed':parsed, 'tree':tree}

def f_pretty(expx):
    return expx['parsed'].pretty()

f_list_flatten = lambda lists: reduce(lambda a,b: a+b, lists)
f_latex_std_abstr = lambda children_out: f_list_flatten((['(', '\\lambda'], [children_out[0]], ['.'], children_out[1:], [')']))
f_latex_std_appl = lambda children_out: f_list_flatten((['['], children_out, [']']))
f_latex_2d_abstr = lambda children_out: f_list_flatten((['\dfrac{'], [children_out[0]], ['}{'], children_out[1:], ['}']))
f_latex_2d_appl = lambda children_out: children_out
g_latex_style_std = {}
g_latex_style_2d = {'abstr':f_latex_2d_abstr, 'appl':f_latex_2d_appl}

def f_latex(expx, style = {}):
    func_abstr = style.get('abstr', f_latex_std_abstr)
    func_appl = style.get('appl', f_latex_std_appl)
    def recurse(node):
        if node['type'] not in ['var']:
            children_out = f_list_flatten([recurse(x) for x in node['kids']])
            if node['type'] == 'abstr':
                return func_abstr(children_out)
            elif node['type'] == 'appl':
                return func_appl(children_out)
        else:
            return [node['name']]
    return ' '.join(recurse(expx['tree']))

def f_disp_latex(expx, style = {}):
    IPython.display.display(IPython.display.Latex('${}$'.format(f_latex(expx, style))))

def test_latex():
    display(IPython.display.Latex('${}$'.format(f_latex(f_expx('[x.[y z]]') ))))
    display(IPython.display.Latex('${}$'.format(f_latex(f_expx('[x.[x z]]'), g_latex_style_2d))))
    f_disp_latex(f_expx('[x.[y z]]'))
    f_disp_latex(f_expx('[x.[y z]]'), g_latex_style_2d)
    
if g_test:
    test_latex()

In [ ]:
def f_tikz_tree(expx):
    transl = {'application':''}
    make_node = lambda content: 'node{{{}}}'.format(transl.get(content, content))
    make_child = lambda content: 'child{{{}}}'.format(transl.get(content, content))
    make_kid_nodes = lambda kids: ' '.join([make_child(kid) for kid in kids])
    def recurse(node, pars=('{', '}')):
        if node['type'] not in ['var']:
            children_out = [recurse(x) for x in node['kids']]
            if node['type'] == 'abstraction':
                return ' '.join([ ''.join([pars[0], children_out[0]]), make_kid_nodes(children_out[1:]), pars[1]])
            else:
                return ' '.join([ ''.join([pars[0], make_node(node['type'])]), make_kid_nodes(children_out), pars[1]])
        else:
            return make_node(node['name'])
    tikz_code = recurse(expx['tree'], ('\\',';'))
    return tikz_code

def f_disp_tikz_tree(expx, style = {}):
    IPython.get_ipython().run_cell('%%tikz\n' + f_tikz_tree(expx))

def test_tree():
    print f_tikz_tree(f_expx('[x.[y z]]'))
    print f_tikz_tree(f_expx('[x y]'))
    print f_tikz_tree(f_expx('[x. y]'))
    print f_tikz_tree(f_expx('[y z]'))
    print f_tikz_tree(f_expx('[x. [y z]]'))
    
if g_test:
    test_tree()
    
if g_test:
    f_disp_tikz_tree(f_expx('[x. [y [z. [z x]]]]'))

In [ ]:
def f_varbinding(expx):
    def recurse(node):
        if 'kids' in node:
            ((l_bind, l_bound, l_free), (r_bind, r_bound, r_free)) = [recurse(x) for x in node['kids']]
            if node['type'] == 'abstr':
                occurs_free = (list(l_free)[0] in r_free)
                return (r_bind | l_free, 
                        r_bound | l_free if occurs_free else r_bound, 
                        r_free - l_free if occurs_free else r_free)
            elif node['type'] == 'appl':
                return (l_bind | r_bind,
                        l_bound | r_bound,
                        (l_free - r_bound) | (r_free - l_bound))
        else:
            return (set(), set(), set(node['name']))
    (bind, bound, free) = recurse(expx['tree'])
    return {'binding':bind, 'bound':bound, 'free':free}
    
def test_varbinding():
    print f_varbinding(f_expx('x'))
    print f_varbinding(f_expx('[x. x]'))
    print f_varbinding(f_expx('[x y]'))
    print f_varbinding(f_expx('[x. [x y]]'))

if g_test:
    test_varbinding()